---
# Safety and Security - Ingest Video Streams

To avoid the need for a set of live cameras for this demo, we play back video from a series of JPEG files on disk
and write each video frame to SDP.

---

### Prerequisites

1. [install_dependencies.ipynb](install_dependencies.ipynb).
2. Download video files from the [VIRAT Video Dataset](https://viratdata.org/) and the [MEVA Video Dataset](http://mevadata.org/).
3. [transcode_video_to_jpeg.ipynb](transcode_video_to_jpeg.ipynb).

### How to use this Notebook
1. Click *Kernel* -> *Restart Kernel and Run All Cells*.

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2
import grpc
import imp
import pravega.grpc_gateway as pravega
import pravega.video as video
from pravega.video import OutputStream, ImageFileSequenceLoader
import os

### Define Pravega stream parameters

In [ ]:
gateway = os.environ['PRAVEGA_GRPC_GATEWAY_ADDRESS']
scope = 'examples'
stream = 'safety-and-security-1'

### Initialize connection to Pravega GRPC Gateway

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
#pravega_client.CreateScope(pravega.pb.CreateScopeRequest(scope=scope))

### Create Pravega stream

In [ ]:
output_stream = OutputStream(pravega_client, scope, stream)
#output_stream.delete_stream()
output_stream.create_stream()

In [ ]:
output_stream.truncate_stream()

### Ingest JPEG images from files

In [ ]:
data_dir = '../../../data/video/'
camera_filespecs = [
    data_dir + 'drone/DJI_0023/*.jpg',
    data_dir + 'virat/aerial/09152008flight2tape1_10/*.jpg',
    data_dir + 'virat/ground/VIRAT_S_000200_01_000226_000268/*.jpg',    
    
    data_dir + 'virat/ground/VIRAT_S_000201_08_001652_001838/*.jpg',
    data_dir + 'drone/DJI_0039/*.jpg',
    data_dir + 'meva/uav-drop-01/2018-03-13/17/2018-03-13.17-30-58.17-40-14.uav1/*.jpg',

    data_dir + 'virat/aerial/09172008flight1tape1_5/*.jpg',
    data_dir + 'virat/aerial/09152008flight2tape3_1/*.jpg',
    data_dir + 'virat/aerial/09152008flight2tape1_7/*.jpg',
]
fps = 10  # drone is 24, virat is 30
step = 3

In [ ]:
imp.reload(video);
import pravega.video as video
from pravega.video import UnindexedStream, OutputStream, IndexedStream, opencv_image_to_mpl, VideoPlayer, ImageFileSequenceLoader
imp.reload(video);

In [ ]:
loader = ImageFileSequenceLoader(scope, stream, camera_filespecs, fps)

In [ ]:
events_to_write = loader.event_generator()
output_stream.write_events(events_to_write)